In [1]:
# Course - Data Wrangling
# Name - Vikas Ranjan
# Final Project - Milestone 3
#                Cleaning/Formatting Website Data

In [99]:
# load libraries 
import pandas as pd
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt

In [100]:
# Read the web page using beautiful soup and fetch data as text
def readWebPage():
    allPages_df = []
    for offset in range(0, 1500, 100):
        # URL is for the web page of yahoo finance to display custom screener. 
        url = 'https://finance.yahoo.com/screener/unsaved/9cf4468e-7dc3-4be3-bc18-1db7ab68efcf?count=100&offset='+str(offset)+''
        # Using get request connect to the URL and fetch webpage data in text format.       
        response = requests.get(url).text
        soup = BeautifulSoup(response)
        singlePage_df = parseData(soup)
        # Append dataframes into a list
        allPages_df.append(singlePage_df)                             
        
    # Concatenate all dataframes of each page into single dataframe        
    finalRawStocks_df = pd.concat(allPages_df, ignore_index=True)      
    
    return finalRawStocks_df

In [101]:
# Read the web page data, fetch and parse the table contents into lists and eventually to the dataframe. 

def parseData(soup):
    
    # Declare the lists to parse and fetch data from each column of the table from the web page
    symbols = []
    names = []
    price_intraday = []
    changes = []
    percent_chg = []
    volume = []
    avg_vol_90 = []
    market_cap = []
    pe_ratio = []
    one_year_range = []
    
    # After performing inspect on the webpage (custom screener on yahoo finance), we first identity for the class name "simpTblRow" 
    # Using aria-label attribute of the html page, we can identify all the column names of the table. 
    # using table data tag, fetch the data for each column in designated lists.
    for row in soup.find_all('tbody'):
        for srow in row.find_all('tr', attrs={'class':'simpTblRow'}):
            for symbol in srow.find_all('td', attrs={'aria-label':'Symbol'}):
                symbols.append(symbol.text)
            for name in srow.find_all('td', attrs={'aria-label':'Name'}):
                names.append(name.text)
            for price in srow.find_all('td', attrs={'aria-label':'Price (Intraday)'}):
                price_intraday.append(price.text)
            for change in srow.find_all('td', attrs={'aria-label':'Change'}):
                changes.append(change.text)
            for perchange in srow.find_all('td', attrs={'aria-label':'% Change'}):
                percent_chg.append(perchange.text)
            for vol in srow.find_all('td', attrs={'aria-label':'Volume'}):
                volume.append(vol.text)
            for avg_vol in srow.find_all('td', attrs={'aria-label':'Avg Vol (3 month)'}):
                avg_vol_90.append(avg_vol.text)
            for market in srow.find_all('td', attrs={'aria-label':'Market Cap'}):
                market_cap.append(market.text)
            for peratio in srow.find_all('td', attrs={'aria-label':'PE Ratio (TTM)'}):
                pe_ratio.append(peratio.text)
            for range in srow.find_all('td', attrs={'aria-label':'52 Week Range'}):
                one_year_range.append(range.text)
    
    # Combine data of all the lists into the dataframe for a single page        
    singlePage_df = pd.DataFrame({"Symbol": symbols, "Name": names, "Price_Intraday": price_intraday, "Change": changes, "Percentage_Change": percent_chg, "Volume": volume, 
              "Avg_Vol_90_Days": avg_vol_90, "Market_Cap": market_cap, "PE_Ratio": pe_ratio, "52_Week_Range": one_year_range})
 
    return singlePage_df

In [102]:
def missing_values_table(finalRawStocks_df):
        zero_val = (finalRawStocks_df == 0.00).astype(int).sum(axis=0)
        mis_val = finalRawStocks_df.isnull().sum()
        mis_val_percent = 100 * finalRawStocks_df.isnull().sum() / len(finalRawStocks_df)
        misval_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        misval_table = misval_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        misval_table['Total Zero Missing Values'] = misval_table['Zero Values'] + misval_table['Missing Values']
        misval_table['% Total Zero Missing Values'] = 100 * misval_table['Total Zero Missing Values'] / len(finalRawStocks_df)
        misval_table['% Total Missing Values'] = 100 * misval_table['Missing Values'] / len(finalRawStocks_df)
        misval_table['Data Type'] = finalRawStocks_df.dtypes
        misval_table = misval_table[
            misval_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(finalRawStocks_df.shape[1]) + " columns and " + str(finalRawStocks_df.shape[0]) + " Rows.\n"      
            "There are " + str(misval_table.shape[0]) +
              " columns that have missing values.")
        print ("There are ", sum(misval_table['% Total Missing Values'] > 50), "columns having greater than 50% missing value.")
        print ("There are ", sum(misval_table['% Total Missing Values'] > 40), "columns having greater than 40% missing value.")
        print ("There are ", sum(misval_table['% Total Missing Values'] > 30), "columns having greater than 30% missing value.")
        print ("There are ", sum(misval_table['% Total Missing Values'] > 20), "columns having greater than 20% missing value.")
        print ("There are ", sum(misval_table['% Total Missing Values'] > 10), "columns having greater than 10% missing value.")

        return misval_table

In [103]:
# Get the stocks data from URL, parse the intended table data and load them into the dataframe. 
finalRawStocks_df = readWebPage() 

In [104]:
# Dispaly first few records
finalRawStocks_df.head()

,Symbol,Name,Price_Intraday,Change,Percentage_Change,Volume,Avg_Vol_90_Days,Market_Cap,PE_Ratio,52_Week_Range
0,HTZ,"Hertz Global Holdings, Inc.",1.7500,-0.0100,-0.57%,29.155M,34.767M,273.361M,N/A,
1,CLNY,"Colony Capital, Inc.",3.2600,+0.0900,+2.84%,18.523M,6.067M,1.572B,N/A,
2,SWN,Southwestern Energy Company,3.1400,+0.0100,+0.32%,14.433M,16.362M,1.901B,N/A,
3,CNK,"Cinemark Holdings, Inc.",9.36,+0.05,+0.54%,10.21M,9.321M,1.117B,N/A,
4,SAVE,"Spirit Airlines, Inc.",18.58,+0.62,+3.45%,10.883M,9.263M,1.815B,N/A,


In [105]:
# Identify the shape of the data frame in terms of numbers of rows and columns. 
finalRawStocks_df.shape

(1221, 10)

In [106]:
# Check for missing values in the stocks dataset
missing_values_table(finalRawStocks_df)

Your selected dataframe has 10 columns and 1221 Rows.
There are 0 columns that have missing values.
There are  0 columns having greater than 50% missing value.
There are  0 columns having greater than 40% missing value.
There are  0 columns having greater than 30% missing value.
There are  0 columns having greater than 20% missing value.
There are  0 columns having greater than 10% missing value.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,% Total Missing Values,Data Type


In [107]:
# Identify and eliminate duplicates
print("Symbol has duplicates - {}".format(any(finalRawStocks_df.Symbol.duplicated())))

Symbol has duplicates - True


In [108]:
# Remove duplicates all other records for each ticker symbol 
cleanStocksData_df = finalRawStocks_df.drop_duplicates(subset=['Symbol'], keep='last')

In [109]:
# Drop na
cleanStocksData_df.dropna()

,Symbol,Name,Price_Intraday,Change,Percentage_Change,Volume,Avg_Vol_90_Days,Market_Cap,PE_Ratio,52_Week_Range
0,HTZ,"Hertz Global Holdings, Inc.",1.7500,-0.0100,-0.57%,29.155M,34.767M,273.361M,N/A,
1,CLNY,"Colony Capital, Inc.",3.2600,+0.0900,+2.84%,18.523M,6.067M,1.572B,N/A,
2,SWN,Southwestern Energy Company,3.1400,+0.0100,+0.32%,14.433M,16.362M,1.901B,N/A,
3,CNK,"Cinemark Holdings, Inc.",9.36,+0.05,+0.54%,10.21M,9.321M,1.117B,N/A,
4,SAVE,"Spirit Airlines, Inc.",18.58,+0.62,+3.45%,10.883M,9.263M,1.815B,N/A,
...,...,...,...,...,...,...,...,...,...,...
1216,OFG-PA,OFG Bancorp PFD A 7.125%,25.89,0.00,0.00%,701,916,1.138B,26.50,
1217,NM-PG,Navios Maritime Holdings Inc.,3.1700,-0.3200,-9.17%,102,N/A,20.081M,N/A,
1218,NM-PH,Navios Maritime Holdings Inc.,2.9995,+0.0095,+0.32%,"4,704",N/A,20.303M,N/A,
1219,NCZ-PA,AllianzGI Convertible & Income Fund II,26.00,+0.01,+0.04%,409,N/A,391.979M,83.60,


In [110]:
# Drop rows that are not of any value for the analysis or have a large % of missing data
cleanStocksData_df = cleanStocksData_df.drop(["52_Week_Range"], axis = 1)

In [111]:
# Describe the dataframe
cleanStocksData_df.describe(include='all',).T

,count,unique,top,freq
Symbol,1194,1194,ANH,1
Name,1194,1072,"Customers Bancorp, Inc.",5
Price_Intraday,1194,961,2.0500,5
Change,1194,237,0.00,53
Percentage_Change,1194,547,0.00%,66
Volume,1194,1176,0,5
Avg_Vol_90_Days,1194,1183,N/A,4
Market_Cap,1194,1150,1.217B,3
PE_Ratio,1194,511,N/A,651


In [112]:
# Since P/E ratio column has most of the values as N/A, we can get rid of it 
cleanStocksData_df = cleanStocksData_df.drop(["PE_Ratio"], axis = 1)

In [113]:
# check the data types of the columns  
cleanStocksData_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1194 entries, 0 to 1220
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Symbol             1194 non-null   object
 1   Name               1194 non-null   object
 2   Price_Intraday     1194 non-null   object
 3   Change             1194 non-null   object
 4   Percentage_Change  1194 non-null   object
 5   Volume             1194 non-null   object
 6   Avg_Vol_90_Days    1194 non-null   object
 7   Market_Cap         1194 non-null   object
dtypes: object(8)
memory usage: 84.0+ KB


In [114]:
# Check the shape of the clean dataframe 
cleanStocksData_df.shape

(1194, 8)

In [115]:
# Get the glimpse of clean dataframe
cleanStocksData_df.head()

,Symbol,Name,Price_Intraday,Change,Percentage_Change,Volume,Avg_Vol_90_Days,Market_Cap
0,HTZ,"Hertz Global Holdings, Inc.",1.7500,-0.0100,-0.57%,29.155M,34.767M,273.361M
1,CLNY,"Colony Capital, Inc.",3.2600,+0.0900,+2.84%,18.523M,6.067M,1.572B
2,SWN,Southwestern Energy Company,3.1400,+0.0100,+0.32%,14.433M,16.362M,1.901B
3,CNK,"Cinemark Holdings, Inc.",9.36,+0.05,+0.54%,10.21M,9.321M,1.117B
4,SAVE,"Spirit Airlines, Inc.",18.58,+0.62,+3.45%,10.883M,9.263M,1.815B


In [116]:
# Extract the clean dataframe to a csv file
cleanStocksData_df.to_csv('Stocks_web.csv', encoding='utf-8', index=False)